<img src="{{site.baseurl}}/images/func1.png"> 

- Meets the requirements because:
    - It defines a function get_emotion that returns book-related emotions.
    - It takes a parameter (title_id) that affects functionality.
    - It implements sequencing (fetching emotions), selection (error handling with try-except), and iteration (for loop to process emotions).

---

<img src="{{site.baseurl}}/images/calltofunc1.png"> 

- Meets the requirements because:
    - It calls the get_emotion function when the “Fetch Emotions” button is clicked.
    - It uses the selected book’s titleId as an argument.
    - It handles errors and updates the UI dynamically.

---

<img src="{{site.baseurl}}/images/list.png"> 

- Meets the requirements because:
    - It defines a list named emotions containing multiple Emotion objects.
    - Each item in the list represents a reaction to a book.

---

<img src="{{site.baseurl}}/images/listinuse.png"> 

- Meets the requirements because:
    - It calls initEmotion(), which uses the emotions list.
    - It ensures all initial data, including emotions, is loaded properly.

---

https://apcentral.collegeboard.org/media/pdf/ap-computer-science-principles-course-and-exam-description.pdf 
